# Feature Engineering

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Load-Libraries" data-toc-modified-id="Load-Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Libraries</a></span></li><li><span><a href="#Load-data-from-Pickle" data-toc-modified-id="Load-data-from-Pickle-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load data from Pickle</a></span></li><li><span><a href="#Split-the-dataset" data-toc-modified-id="Split-the-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Split the dataset</a></span></li><li><span><a href="#List-of-features-to-drop" data-toc-modified-id="List-of-features-to-drop-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>List of features to drop</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#Pickle-the-CatBoost-Features" data-toc-modified-id="Pickle-the-CatBoost-Features-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Pickle the CatBoost Features</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>LGBM</a></span></li><li><span><a href="#Pickle-the-LGBM-features" data-toc-modified-id="Pickle-the-LGBM-features-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Pickle the LGBM features</a></span></li></ul></li><li><span><a href="#Next-Step---Modeling" data-toc-modified-id="Next-Step---Modeling-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Next Step - Modeling</a></span></li></ul></div>

## Introduction
This is where the bulk of the feature engineering takes place. The code can be found under `src/fe_modeling.py`. Here we are using feature aggregation to capture the historical activity of the transactions. Each transaction information about previous transactions including aggregations so that we can track the mean and standard deviations for specific time intervals. These properties should allow our models to perform a `behavioral analysis` of the activities of each credit card. Of course, one of the side effects of this is that the feature space can grow considerabily. <p></p>
We are also `frequency encoding` values, doing `min-max normalizaton`, clipping negative values, and treating `NaNs`.

## Load Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import sys
import gc
import pickle

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.fe_modeling import *

%matplotlib inline

# Some Pandas options so that we can examine all the columns in the dataset with head
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000
pd.options.display.max_colwidth = 1000

In [3]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [4]:
SEED = 42
seed_everything(SEED)
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
TARGET = 'isFraud'
NFOLDS = 5

## Load data from Pickle
We have already done some cleaning and feature engineering on the datasets. We then save the datasets to pickle files. Here, we load the datasets from pickle so that we can continue working.

In [5]:
%%time
X = pd.read_pickle('../data/train_reduced.pkl')
X_test = pd.read_pickle('../data/test_reduced.pkl')

y = X[TARGET]
X = X.drop(TARGET, axis=1)
        
print(f'X.shape : {X.shape}, X_test.shape : {X_test.shape}')

X.shape : (590540, 434), X_test.shape : (506691, 434)
Wall time: 7.41 s


## Split the dataset
As per usual, we split our data into training and validation.

In [6]:
def train_val_split_by_time(X, y, test_size=0.2):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, shuffle=False)
    
    print(f'train.shape: {X_train.shape}, val.shape: {X_val.shape}')
    
    return X_train, y_train, X_val, y_val

In [7]:
X_train, y_train, X_val, y_val = train_val_split_by_time(X, y)

train.shape: (472432, 434), val.shape: (118108, 434)


## List of features to drop
This list contains temporary features that will be created during the feature engineering stage. They are now dropped as they are longer needed.

In [9]:
cols_to_drop = [
    'D5_DT_W_std_score',
    'ProductCD_TransactionAmt_DT_W',
    'D4_DT_D_std_score',
    'D15_DT_D_std_score',
    'D3_DT_W_std_score',
    'D11_DT_W_std_score',
    'card3_card5_DT_W_week_day_dist',
    'card5_DT_W_week_day_dist',
    'D10_DT_D_std_score',
    'card3_card5_DT_D',
    'ProductCD_cents_DT_D',
    'D4_DT_W_std_score',
    'D15_DT_W_std_score',
    'uid_DT_D',
    'card3_DT_W_week_day_dist',
    'D10_DT_W_std_score',
    'D8_DT_D_std_score',
    'card3_card5_DT_W',
    'ProductCD_cents_DT_W',
    'uid_DT_W',
    'D8_DT_W_std_score'
]

## Feature Engineering
This function does the bulk of the feature engineering. These are the most important tasks:
* Identify valid credit cards - those cards that have more than two entries.
* Create timeblocks for use in feature aggregation to examine transactions based on periodic behavior.
* Use feature aggregation to capture behavioral aspects of card transactions.
* Feature encoding.
* Removing temporary and unnecessary features.

The datasets are a bit different for `CatBoost` and `LGBM`, so we do feature engineering twice. In the future I will do the majority of the work mutual to both algorithms in one pass, and that which is only needed for `CatBoost` separately. Which can then be merged to a copy of the first for `CatBoost`.


### CatBoost 

In [13]:
X_train_fe, X_val_fe, feature_cols1, category_cols1 = fe1(X_train, X_val, cols_to_drop, algo='CatBoost')

Rare data card1 5134
No intersection in Train card1 20399
Intersection in Train card1 452033
####################
Rare data ProductCD_card1 10509
No intersection in Train ProductCD_card1 33115
Intersection in Train ProductCD_card1 439317
####################
Rare data card1_addr1 21640
No intersection in Train card1_addr1 57867
Intersection in Train card1_addr1 414565
####################
Rare data TransactionAmt_dist2 18260
No intersection in Train TransactionAmt_dist2 49343
Intersection in Train TransactionAmt_dist2 423089
####################
No intersection in Train card2 6102
Intersection in Train card2 466330
####################
No intersection in Train card3 146
Intersection in Train card3 472286
####################
No intersection in Train card4 0
Intersection in Train card4 472432
####################
No intersection in Train card5 7339
Intersection in Train card5 465093
####################
No intersection in Train card6 45
Intersection in Train card6 472387
###############

uid_aggregation: card5_id_14_mean
uid_aggregation: card5_id_14_std
uid_aggregation: uid_id_14_mean
uid_aggregation: uid_id_14_std
uid_aggregation: card3_card5_id_14_mean
uid_aggregation: card3_card5_id_14_std
uid_aggregation: uid_V258_mean
uid_aggregation: uid_V258_std
uid_aggregation: card3_card5_V258_mean
uid_aggregation: card3_card5_V258_std
uid_aggregation: uid_V306_mean
uid_aggregation: uid_V306_std
uid_aggregation: card3_card5_V306_mean
uid_aggregation: card3_card5_V306_std
uid_aggregation: uid_V307_mean
uid_aggregation: uid_V307_std
uid_aggregation: card3_card5_V307_mean
uid_aggregation: card3_card5_V307_std
uid_aggregation: uid_V308_mean
uid_aggregation: uid_V308_std
uid_aggregation: card3_card5_V308_mean
uid_aggregation: card3_card5_V308_std
uid_aggregation: uid_V294_mean
uid_aggregation: uid_V294_std
uid_aggregation: card3_card5_V294_mean
uid_aggregation: card3_card5_V294_std
timeblock frequency encoding: ProductCD_TransactionAmt_DT_D
timeblock frequency encoding: ProductCD_T

processing nan group agg for: ['V143', 'V144', 'V145', 'V150', 'V151', 'V152', 'V159', 'V160', 'V164', 'V165', 'V166']
processing nan group agg for: ['V167', 'V168', 'V172', 'V173', 'V176', 'V177', 'V178', 'V179', 'V181', 'V182', 'V183', 'V186', 'V187', 'V190', 'V191', 'V192', 'V193', 'V196', 'V199', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216']
processing nan group agg for: ['V169', 'V170', 'V171', 'V174', 'V175', 'V180', 'V184', 'V185', 'V188', 'V189', 'V194', 'V195', 'V197', 'V198', 'V200', 'V201', 'V208', 'V209', 'V210']
processing nan group agg for: ['V217', 'V218', 'V219', 'V223', 'V224', 'V225', 'V226', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V235', 'V236', 'V237', 'V240', 'V241', 'V242', 'V243', 'V244', 'V246', 'V247', 'V248', 'V249', 'V252', 'V253', 'V254', 'V257', 'V258', 'V260', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V269', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278']
processing nan group

Column: V295  | Dominator: 0.0
Column: V296  | Dominator: 0.0
Column: V297  | Dominator: 0.0
Column: V298  | Dominator: 0.0
Column: V299  | Dominator: 0.0
Column: V300  | Dominator: 0.0
Column: V301  | Dominator: 0.0
Column: V305  | Dominator: 1.0
Column: V309  | Dominator: 0.0
Column: V311  | Dominator: 0.0
Column: V316  | Dominator: 0.0
Column: V317  | Dominator: 0.0
Column: V318  | Dominator: 0.0
Column: V319  | Dominator: 0.0
Column: V320  | Dominator: 0.0
Column: V321  | Dominator: 0.0
Column: V322  | Dominator: -999.0
Column: V323  | Dominator: -999.0
Column: V324  | Dominator: -999.0
Column: V325  | Dominator: -999.0
Column: V326  | Dominator: -999.0
Column: V327  | Dominator: -999.0
Column: V328  | Dominator: -999.0
Column: V329  | Dominator: -999.0
Column: V330  | Dominator: -999.0
Column: V331  | Dominator: -999.0
Column: V332  | Dominator: -999.0
Column: V333  | Dominator: -999.0
Column: V334  | Dominator: -999.0
Column: V335  | Dominator: -999.0
Column: V336  | Dominator: -

Column: nan_group_508595  | Dominator: 1
Column: nan_group_508589  | Dominator: 1
Column: nan_group_12  | Dominator: 0
Column: nan_group_508189  | Dominator: 1
Column: nan_group_524216  | Dominator: 1
Column: nan_group_585385  | Dominator: 1
Column: nan_group_100  | Dominator: 0
Column: nan_group_519723  | Dominator: 1
Column: nan_group_559  | Dominator: 0
Column: nan_group_781  | Dominator: 0
Column: nan_group_225  | Dominator: 0
Column: nan_group_13087  | Dominator: 0
Column: nan_group_182  | Dominator: 0
Column: nan_group_187  | Dominator: 0
Column: nan_group_8310  | Dominator: 0
Column: nan_group_8325  | Dominator: 0
Column: nan_group_56771  | Dominator: 0
Column: nan_group_77261  | Dominator: 0
Column: nan_group_13160  | Dominator: 0
Column: nan_group_15575  | Dominator: 0
Column: nan_group_160  | Dominator: 0
Column: nan_group_573  | Dominator: 0
Duplicate card3_FE_FULL
Duplicate card3_TransactionAmt_mean
Duplicate card3_TransactionAmt_std
Duplicate card3_id_01_mean
Duplicate car

In [15]:
X_train_fe.head()

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,M1,M2,M3,M4,M5,M6,M7,M8,M9,id_01,id_02,id_03,id_05,id_06,id_07,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_32,id_34,id_35,id_36,id_37,id_38,DeviceType,uid,is_december,card1_FE_FULL,card2_FE_FULL,card5_FE_FULL,ProductCD_card1_FE_FULL,card1_addr1_FE_FULL,TransactionAmt_dist2_FE_FULL,uid_FE_FULL,card3_card5,card3_DT_D_hour_dist,card3_DT_D_hour_dist_best,card3_DT_W_week_day_dist_best,card5_DT_D_hour_dist,card5_DT_D_hour_dist_best,card5_DT_W_week_day_dist_best,card3_card5_DT_D_hour_dist,card3_card5_DT_D_hour_dist_best,card3_card5_DT_W_week_day_dist_best,uid_D1_mean,uid_D1_std,card3_card5_D1_mean,card3_card5_D1_std,uid_D2_mean,uid_D2_std,card3_card5_D2_mean,card3_card5_D2_std,uid_D3_mean,uid_D3_std,card3_card5_D3_mean,card3_card5_D3_std,uid_D4_mean,uid_D4_std,card3_card5_D4_mean,card3_card5_D4_std,uid_D5_mean,uid_D5_std,card3_card5_D5_mean,card3_card5_D5_std,uid_D6_mean,uid_D6_std,card3_card5_D6_mean,card3_card5_D6_std,uid_D7_mean,uid_D7_std,card3_card5_D7_mean,card3_card5_D7_std,uid_D8_mean,uid_D8_std,card3_card5_D8_mean,card3_card5_D8_std,uid_D9_mean,card3_card5_D9_mean,card3_card5_D9_std,uid_D10_mean,uid_D10_std,card3_card5_D10_mean,card3_card5_D10_std,uid_D11_mean,uid_D11_std,card3_card5_D11_mean,card3_card5_D11_std,uid_D12_mean,uid_D12_std,card3_card5_D12_mean,card3_card5_D12_std,uid_D13_mean,uid_D13_std,card3_card5_D13_mean,card3_card5_D13_std,uid_D14_mean,uid_D14_std,card3_card5_D14_mean,card3_card5_D14_std,uid_D15_mean,uid_D15_std,card3_card5_D15_mean,card3_card5_D15_std,D8_not_same_day,D3_DT_D_min_max,D3_DT_D_std_score,D4_DT_D_min_max,D5_DT_D_min_max,D5_DT_D_std_score,D10_DT_D_min_max,D11_DT_D_min_max,D11_DT_D_std_score,D15_DT_D_min_max,D3_DT_W_min_max,D4_DT_W_min_max,D5_DT_W_min_max,D10_DT_W_min_max,D11_DT_W_min_max,D15_DT_W_min_max,D1_scaled,D2_scaled,D1_FE_FULL,D2_FE_FULL,D3_FE_FULL,D4_FE_FULL,D5_FE_FULL,D10_FE_FULL,D11_FE_FULL,D15_FE_FULL,TransactionAmt_check,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card5_TransactionAmt_mean,card5_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,card3_card5_TransactionAmt_mean,card3_card5_TransactionAmt_std,TransactionAmt_DT_D_min_max,TransactionAmt_DT_D_std_score,TransactionAmt_DT_W_min_max,TransactionAmt_DT_W_std_score,card1_id_01_mean,card1_id_01_std,card2_id_01_mean,card2_id_01_std,card5_id_01_mean,card5_id_01_std,uid_id_01_mean,uid_id_01_std,card3_card5_id_01_mean,card3_card5_id_01_std,card1_id_02_mean,card1_id_02_std,card2_id_02_mean,card2_id_02_std,card5_id_02_mean,card5_id_02_std,uid_id_02_mean,uid_id_02_std,card3_card5_id_02_mean,card3_card5_id_02_std,card1_id_05_mean,card1_id_05_std,card2_id_05_mean,card2_id_05_std,card5_id_05_mean,card5_id_05_std,uid_id_05_mean,uid_id_05_std,card3_card5_id_05_mean,card3_card5_id_05_std,card1_id_06_mean,card1_id_06_std,card2_id_06_mean,card2_id_06_std,card3_id_06_std,card5_id_06_mean,card5_id_06_std,uid_id_06_mean,uid_id_06_std,card3_card5_id_06_mean,card3_card5_id_06_std,card1_id_09_mean,card1_id_09_std,card2_id_09_mean,card2_id_09_std,card5_id_09_mean,card5_id_09_std,uid_id_09_mean,card3_card5_id_09_mean,card3_card5_id_09_std,card1_id_14_mean,card1_id_14_std,card2_id_14_mean,card2_id_14_std,card5_id_14_mean,card5_id_14_std,uid_id_14_mean,uid_id_14_std,card3_card5_id_14_mean,card3_card5_id_14_std,uid_V258_mean,uid_V258_std,card3_card5_V258_mean,card3_card5_V258_std,uid_V306_mean,uid_V306_std,card3_card5_V306_mean,card3_card5_V306_std,uid_V307_mean,uid_V307_std,card3_card5_V307_mean,card3_card5_V307_std,uid_V308_mean,uid_V308_std,card3_card5_V308_mean,card3_card5_V308_std,uid_V294_mean,uid_V294_std,card3_card5_V294_mean,card3_card5_V294_std,ProductCD_TransactionAmt_DT_D,ProductCD_TransactionAmt_FE_FULL,ProductCD_cents_FE_FU

### Pickle the CatBoost Features

In [16]:
X_train_fe.to_pickle('../data/X_train_fe_engineered_cat.pkl')
y_train.to_pickle('../data/y_train_fe_engineered_cat.pkl')

X_val_fe.to_pickle('../data/X_val_fe_engineered_cat.pkl')
y_val.to_pickle('../data/y_val_fe_engineered_cat.pkl')

with open('../data/cat_cols_fe_engineered_cat.pkl', 'wb') as f:
    pickle.dump(category_cols1, f)
    
with open('../data/feat_cols_fe_engineered_cat.pkl', 'wb') as f:
    pickle.dump(feature_cols1, f)

### LGBM

In [11]:
X_train, X_val, category_cols1 = fe1(X_train, X_val, cols_to_drop, algo='LGBM')

Rare data card1 5134
No intersection in Train card1 20399
Intersection in Train card1 452033
####################
Rare data ProductCD_card1 10509
No intersection in Train ProductCD_card1 33115
Intersection in Train ProductCD_card1 439317
####################
Rare data card1_addr1 21640
No intersection in Train card1_addr1 57867
Intersection in Train card1_addr1 414565
####################
Rare data TransactionAmt_dist2 18260
No intersection in Train TransactionAmt_dist2 49343
Intersection in Train TransactionAmt_dist2 423089
####################
No intersection in Train card2 6102
Intersection in Train card2 466330
####################
No intersection in Train card3 146
Intersection in Train card3 472286
####################
No intersection in Train card4 0
Intersection in Train card4 472432
####################
No intersection in Train card5 7339
Intersection in Train card5 465093
####################
No intersection in Train card6 45
Intersection in Train card6 472387
###############

uid_aggregation: card5_id_14_mean
uid_aggregation: card5_id_14_std
uid_aggregation: uid_id_14_mean
uid_aggregation: uid_id_14_std
uid_aggregation: card3_card5_id_14_mean
uid_aggregation: card3_card5_id_14_std
uid_aggregation: uid_V258_mean
uid_aggregation: uid_V258_std
uid_aggregation: card3_card5_V258_mean
uid_aggregation: card3_card5_V258_std
uid_aggregation: uid_V306_mean
uid_aggregation: uid_V306_std
uid_aggregation: card3_card5_V306_mean
uid_aggregation: card3_card5_V306_std
uid_aggregation: uid_V307_mean
uid_aggregation: uid_V307_std
uid_aggregation: card3_card5_V307_mean
uid_aggregation: card3_card5_V307_std
uid_aggregation: uid_V308_mean
uid_aggregation: uid_V308_std
uid_aggregation: card3_card5_V308_mean
uid_aggregation: card3_card5_V308_std
uid_aggregation: uid_V294_mean
uid_aggregation: uid_V294_std
uid_aggregation: card3_card5_V294_mean
uid_aggregation: card3_card5_V294_std
timeblock frequency encoding: ProductCD_TransactionAmt_DT_D
timeblock frequency encoding: ProductCD_T

processing nan group agg for: ['V143', 'V144', 'V145', 'V150', 'V151', 'V152', 'V159', 'V160', 'V164', 'V165', 'V166']
processing nan group agg for: ['V167', 'V168', 'V172', 'V173', 'V176', 'V177', 'V178', 'V179', 'V181', 'V182', 'V183', 'V186', 'V187', 'V190', 'V191', 'V192', 'V193', 'V196', 'V199', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216']
processing nan group agg for: ['V169', 'V170', 'V171', 'V174', 'V175', 'V180', 'V184', 'V185', 'V188', 'V189', 'V194', 'V195', 'V197', 'V198', 'V200', 'V201', 'V208', 'V209', 'V210']
processing nan group agg for: ['V217', 'V218', 'V219', 'V223', 'V224', 'V225', 'V226', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V235', 'V236', 'V237', 'V240', 'V241', 'V242', 'V243', 'V244', 'V246', 'V247', 'V248', 'V249', 'V252', 'V253', 'V254', 'V257', 'V258', 'V260', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V269', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278']
processing nan group

In [12]:
X_train.head(3)

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,M1,M2,M3,M4,M5,M6,M7,M8,M9,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_32,id_34,id_35,id_36,id_37,id_38,DeviceType,is_december,card1_FE_FULL,card2_FE_FULL,card3_FE_FULL,card5_FE_FULL,ProductCD_card1_FE_FULL,card1_addr1_FE_FULL,TransactionAmt_dist2_FE_FULL,uid_FE_FULL,card3_DT_D_hour_dist,card3_DT_D_hour_dist_best,card3_DT_W_week_day_dist_best,card5_DT_D_hour_dist,card5_DT_D_hour_dist_best,card5_DT_W_week_day_dist_best,card3_card5_DT_D_hour_dist,card3_card5_DT_D_hour_dist_best,card3_card5_DT_W_week_day_dist_best,uid_D1_mean,uid_D1_std,card3_card5_D1_mean,card3_card5_D1_std,uid_D2_mean,uid_D2_std,card3_card5_D2_mean,card3_card5_D2_std,uid_D3_mean,uid_D3_std,card3_card5_D3_mean,card3_card5_D3_std,uid_D4_mean,uid_D4_std,card3_card5_D4_mean,card3_card5_D4_std,uid_D5_mean,uid_D5_std,card3_card5_D5_mean,card3_card5_D5_std,uid_D6_mean,uid_D6_std,card3_card5_D6_mean,card3_card5_D6_std,uid_D7_mean,uid_D7_std,card3_card5_D7_mean,card3_card5_D7_std,uid_D8_mean,uid_D8_std,card3_card5_D8_mean,card3_card5_D8_std,uid_D9_mean,uid_D9_std,card3_card5_D9_mean,card3_card5_D9_std,uid_D10_mean,uid_D10_std,card3_card5_D10_mean,card3_card5_D10_std,uid_D11_mean,uid_D11_std,card3_card5_D11_mean,card3_card5_D11_std,uid_D12_mean,uid_D12_std,card3_card5_D12_mean,card3_card5_D12_std,uid_D13_mean,uid_D13_std,card3_card5_D13_mean,card3_card5_D13_std,uid_D14_mean,uid_D14_std,card3_card5_D14_mean,card3_card5_D14_std,uid_D15_mean,uid_D15_std,card3_card5_D15_mean,card3_card5_D15_std,D9_not_na,D8_not_same_day,D8_D9_decimal_dist,D3_DT_D_min_max,D3_DT_D_std_score,D4_DT_D_min_max,D5_DT_D_min_max,D5_DT_D_std_score,D6_DT_D_min_max,D6_DT_D_std_score,D7_DT_D_min_max,D7_DT_D_std_score,D8_DT_D_min_max,D10_DT_D_min_max,D11_DT_D_min_max,D11_DT_D_std_score,D12_DT_D_min_max,D12_DT_D_std_score,D13_DT_D_min_max,D13_DT_D_std_score,D14_DT_D_min_max,D14_DT_D_std_score,D15_DT_D_min_max,D3_DT_W_min_max,D4_DT_W_min_max,D5_DT_W_min_max,D6_DT_W_min_max,D6_DT_W_std_score,D7_DT_W_min_max,D7_DT_W_std_score,D8_DT_W_min_max,D10_DT_W_min_max,D11_DT_W_min_max,D12_DT_W_min_max,D12_DT_W_std_score,D13_DT_W_min_max,D13_DT_W_std_score,D14_DT_W_min_max,D14_DT_W_std_score,D15_DT_W_min_max,D1_scaled,D2_scaled,D1_FE_FULL,D2_FE_FULL,D3_FE_FULL,D4_FE_FULL,D5_FE_FULL,D6_FE_FULL,D7_FE_FULL,D8_FE_FULL,D9_FE_FULL,D10_FE_FULL,D11_FE_FULL,D12_FE_FULL,D13_FE_FULL,D14_FE_FULL,D15_FE_FULL,TransactionAmt_check,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,card3_TransactionAmt_std,card5_TransactionAmt_mean,card5_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,card3_card5_TransactionAmt_mean,card3_card5_TransactionAmt_std,TransactionAmt_DT_D_min_max,TransactionAmt_DT_D_std_score,TransactionAmt_DT_W_min_max,TransactionAmt_DT_W_std_score,card1_id_01_mean,card1_id_01_std,card2_id_01_mean,card2_id_01_std,card3_id_01_mean,card3_id_01_std,card5_id_01_mean,card5_id_01_std,uid_id_01_mean,uid_id_01_std,card3_card5_id_01_mean,card3_card5_id_01_std,card1_id_02_mean,card1_id_02_std,card2_id_02_mean,card2_id_02_std,card3_id_02_mean,card3_id_02_std,card5_id_02_mean,card5_id_02_std,uid_id_02_mean,uid_id_02_std,card3_card5_id_02_mean,card3_card5_id_02_std,card1_id_05_mean,card1_id_05_std,card2_id_05_mean,card2_id_05_std,card3_id_05_mean,card3_id_05_std,card5_id_05_mean,card5_id_05_std,uid_id_05_mean,uid_id_05_std,card3_card5_id_05_mean,card3_card5_id_05_std,card1_id_06_mean,card1_id_06_std,card2_id_06_mean,card2_id_06_std,card3_id_06_mean,card3_id_06_std,card5_id_06_mean,card5_id_06_std,uid_id_06_mean,uid_id_06_std,card3_card5_id_06_mean,card3_card5_id_06_std,card1_id_09_mean,card1_id_09_std,card2_id_09_mean,card2_id_09_std,card3_id_09_mean,card

### Pickle the LGBM features
We pickle the datasets to be unpackaged later for use in modeling.

In [16]:
X_train.to_pickle('../data/X_train_fe_engineered.pkl')
y_train.to_pickle('../data/y_train_fe_engineered.pkl')

X_val.to_pickle('../data/X_val_fe_engineered.pkl')
y_val.to_pickle('../data/y_val_fe_engineered.pkl')

with open('../data/cat_cols_fe_engineered.pkl', 'wb') as f:
    pickle.dump(category_cols1, f)

## Next Step - Modeling